### First time installing elasticsearch
I followed these directions:
https://www.elastic.co/guide/en/elasticsearch/reference/current/deb.html

I have to be root in order to see the logs 'sudo -i'

From the time I ran the startup command to the time I ran the test GET and it worked, it seemed like there was some lag. Verify it is up and running by looking at the logs.

This blog post was helpful: https://bitquabit.com/post/having-fun-python-and-elasticsearch-part-1/

Another blog post Star Wars API: https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/

http://swapi.co/

#### Further Reading
https://blog.insightdatascience.com/anatomy-of-an-elasticsearch-cluster-part-i-7ac9a13b05db#.9c5sp165a

https://elasticsearch-py.readthedocs.io/en/master/

https://www.elastic.co/guide/en/elasticsearch/reference/5.1/query-dsl-multi-match-query.html#phrase-fuzziness

https://en.wikipedia.org/wiki/Elasticsearch

https://github.com/elastic/elasticsearch

#### connect to the cluster

In [1]:
import elasticsearch
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}])

#### make sure Elasticsearch is up and running

In [2]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

{
  "name" : "_84yDSt",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "Wjcm_kHmSmSv2_lBJYQZ6g",
  "version" : {
    "number" : "5.1.2",
    "build_hash" : "c8c4c16",
    "build_date" : "2017-01-11T20:18:39.146Z",
    "build_snapshot" : false,
    "lucene_version" : "6.3.0"
  },
  "tagline" : "You Know, for Search"
}



#### Indexing test data

In [ ]:
es.index(index='posts', doc_type='blog', id=1, body={
    'author': 'Santa Clause',
    'blog': 'Slave Based Shippers of the North',
    'title': 'Using Celery for distributing gift dispatch',
    'topics': ['slave labor', 'elves', 'python',
               'celery', 'antigravity reindeer'],
    'awesomeness': 0.2
})

#### retrieve test data

In [15]:
es.search(index="posts", body={"query": {"match": {'author':'Santa Clause'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'posts',
    u'_score': 0.51623213,
    u'_source': {u'author': u'Santa Clause',
     u'awesomeness': 0.2,
     u'blog': u'Slave Based Shippers of the North',
     u'title': u'Using Celery for distributing gift dispatch',
     u'topics': [u'slave labor',
      u'elves',
      u'python',
      u'celery',
      u'antigravity reindeer']},
    u'_type': u'blog'}],
  u'max_score': 0.51623213,
  u'total': 1},
 u'timed_out': False,
 u'took': 3}

#### Delete test data

In [16]:
es.delete(index='posts', doc_type='blog', id=1)

{u'_id': u'1',
 u'_index': u'posts',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 2,
 u'found': True,
 u'result': u'deleted'}

#### Get Star Wars data

In [3]:
# let's iterate over swapi people documents and index them
import json

r = requests.get('http://localhost:9200') 
i = 1

while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

print(i)

18


Person 17 is returning a 404

Please, notice that we automatically created an index “sw” and a “doc_type” with the indexing command. We get 17 responses from swapi and index them with ES.

In [4]:
# we try to get the document with id=5.
es.get(index='sw', doc_type='people', id=5)

{u'_id': u'5',
 u'_index': u'sw',
 u'_source': {u'birth_year': u'19BBY',
  u'created': u'2014-12-10T15:20:09.791000Z',
  u'edited': u'2014-12-20T21:17:50.315000Z',
  u'eye_color': u'brown',
  u'films': [u'http://swapi.co/api/films/6/',
   u'http://swapi.co/api/films/3/',
   u'http://swapi.co/api/films/2/',
   u'http://swapi.co/api/films/1/',
   u'http://swapi.co/api/films/7/'],
  u'gender': u'female',
  u'hair_color': u'brown',
  u'height': u'150',
  u'homeworld': u'http://swapi.co/api/planets/2/',
  u'mass': u'49',
  u'name': u'Leia Organa',
  u'skin_color': u'light',
  u'species': [u'http://swapi.co/api/species/1/'],
  u'starships': [],
  u'url': u'http://swapi.co/api/people/5/',
  u'vehicles': [u'http://swapi.co/api/vehicles/30/']},
 u'_type': u'people',
 u'_version': 2,
 u'found': True}

#### I ran this twice.... are there duplicates now? No

In [14]:
r = requests.get('http://localhost:9200')
i = 18

while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

print(i)

90


Where is Darth Vader?

In [17]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'4',
    u'_index': u'sw',
    u'_score': 4.4691906,
    u'_source': {u'birth_year': u'41.9BBY',
     u'created': u'2014-12-10T15:18:20.704000Z',
     u'edited': u'2014-12-20T21:17:50.313000Z',
     u'eye_color': u'yellow',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/'],
     u'gender': u'male',
     u'hair_color': u'none',
     u'height': u'202',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'136',
     u'name': u'Darth Vader',
     u'skin_color': u'white',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/13/'],
     u'url': u'http://swapi.co/api/people/4/',
     u'vehicles': []},
    u'_type': u'people'},
   {u'_id': u'44',
    u'_index': u'sw',
    u'_score': 1.919345,
    u'_source': {u'birth_year

This will give us both Darth Vader AND Darth Maul. Id 4 and id 44 (notice that they are in the same index, even if we use different node client call the index command). Both results have a score, although Darth Vader is much higher than Darth Maul (2.77 vs 0.60) since Vader is a exact match. Take that Darth Maul! 

#### Question: How does Elastic Search calculate the score?

So, this query will give us results if the word is contained exactly in our indexed data. What if we want to build some kind of autocomplete input where we get the names that contain the characters we are typing?

There are many ways to do that and another great number of queries.

See: https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html

I picked this one to get all documents with prefix “lu” in their name field: 

We will get Luke Skywalker and Luminara Unduli, both with the same 1.0 score, since they match with the same 2 initial characters. 

In [7]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'sw',
    u'_score': 1.0,
    u'_source': {u'birth_year': u'19BBY',
     u'created': u'2014-12-09T13:50:51.644000Z',
     u'edited': u'2014-12-20T21:17:56.891000Z',
     u'eye_color': u'blue',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/',
      u'http://swapi.co/api/films/7/'],
     u'gender': u'male',
     u'hair_color': u'blond',
     u'height': u'172',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'77',
     u'name': u'Luke Skywalker',
     u'skin_color': u'fair',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/12/',
      u'http://swapi.co/api/starships/22/'],
     u'url': u'http://swapi.co/api/people/1/',
     u'vehicles': [u'http://swapi.co/api/vehicles/14/',
      u'http:

we want to get all elements similar in some way, for a related or correction search we can use something like this:

See: https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html

Fuzziness here is the The maximum edit distance.
https://www.elastic.co/guide/en/elasticsearch/reference/5.1/common-options.html#fuzziness

In [33]:
es.search(
    index="sw", 
    body={
        "query": {
            "multi_match" : { 
                "query" : "jaba",
                "fields" : ["name", "species"],
                "fuzziness" : 1
            }
        }
    }
)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'16',
    u'_index': u'sw',
    u'_score': 1.3984017,
    u'_source': {u'birth_year': u'600BBY',
     u'created': u'2014-12-10T17:11:31.638000Z',
     u'edited': u'2014-12-20T21:17:50.338000Z',
     u'eye_color': u'orange',
     u'films': [u'http://swapi.co/api/films/4/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/1/'],
     u'gender': u'hermaphrodite',
     u'hair_color': u'n/a',
     u'height': u'175',
     u'homeworld': u'http://swapi.co/api/planets/24/',
     u'mass': u'1,358',
     u'name': u'Jabba Desilijic Tiure',
     u'skin_color': u'green-tan, brown',
     u'species': [u'http://swapi.co/api/species/5/'],
     u'starships': [],
     u'url': u'http://swapi.co/api/people/16/',
     u'vehicles': []},
    u'_type': u'people'}],
  u'max_score': 1.3984017,
  u'total': 1},
 u'timed_out': False,
 u'took': 33}